In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [68]:
from fastai.vision.all import *
path = Path("../input/pnevmoniya/train")

# p = Path("../input/pnevmoniya/train")
# p.ls()

In [3]:
# def is_cat(x): return x[0].isupper()
# dls = ImageDataLoaders.from_name_func(
#     p, get_image_files(p), get_y=parent_label ,seed=42,item_tfms=Resize(224))
block = DataBlock(
    blocks =(ImageBlock, CategoryBlock),
    get_items = get_image_files,
    splitter = RandomSplitter(valid_pct=0.2,seed=42),
    get_y = parent_label,
    item_tfms = Resize(224)
)

dls = block.dataloaders(path)

In [6]:
verify_images(path)

In [7]:
path.ls()

In [8]:
dls.train.show_batch(max_n=16,nrows=4)

In [53]:
dls

In [9]:
resnet18 = vision_learner(dls, resnet18, metrics=accuracy)
resnet18.fine_tune(2)

In [16]:
# dls.train.show_batch(max_n=16,nrows=4)

In [11]:
classs = ClassificationInterpretation.from_learner(resnet18)
classs.plot_confusion_matrix()

In [18]:
resnet18.lr_find()

In [13]:
classs.plot_top_losses(50)

In [20]:
from ipywidgets import * 

In [24]:
img = PILImage.create(upload.data[-1])
pred,pred_id,probs = resnet18.predict(img)
print(f"Prediction: {pred}")
print(f"Probability: {probs[pred_id]*100:.1f}%")
img

In [29]:
# def is_cat(x): return x[0].isupper()
# dls = ImageDataLoaders.from_name_func(
#     path, get_image_files(path), valid_pct=0.2, seed=42,
#     get_y=parent_label, label_func=is_cat, 
#     item_tfms=Resize(224))
# learn = vision_learner(dls, resnet18, metrics=accuracy, normalize=False)
# learn.fine_tune(2)

In [69]:
pen = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [70]:
dls = pen.dataloaders(path)

In [71]:
dls.valid.show_batch(max_n=10, nrows=2)

In [18]:
# pen = pen.new(item_tfms=Resize(128, ResizeMethod.Squish))
# dls = pen.dataloaders(p)
# dls.valid.show_batch(max_n=10, nrows=2)

In [74]:
resnet50 = cnn_learner(dls,resnet50,metrics=accuracy)
resnet50.fine_tune(2)

In [75]:
resnet50.lr_find()

In [76]:
classs.plot_top_losses(10,nrows=2)

In [77]:
classs = ClassificationInterpretation.from_learner(resnet50)
classs.plot_confusion_matrix()

In [72]:
resnet34 = vision_learner(dls, resnet34, metrics=accuracy)
resnet34.fine_tune(3)

In [79]:
classs = ClassificationInterpretation.from_learner(resnet34)
classs.plot_confusion_matrix()

In [35]:
upload = widgets.FileUpload()
upload

In [81]:
img = PILImage.create(upload.data[-1])
pred,pred_id,probs = resnet50.predict(img)
print(f"Prediction: {pred}")
print(f"Probability: {probs[pred_id]*100:.1f}%")
img

In [80]:
img = PILImage.create(upload.data[-1])
pred,pred_id,probs = resnet34.predict(img)
print(f"Prediction: {pred}")
print(f"Probability: {probs[pred_id]*100:.1f}%")
img

In [41]:
pen2 = pen.new(item_tfms=Resize(128),
    batch_tfms=aug_transforms(mult=2))
dls = pen2.dataloaders(path)
dls.train.show_batch(max_n=12, nrows=4, unique=False)

In [83]:
vgg19_bn = vision_learner(dls, vgg19_bn, metrics=accuracy)
vgg19_bn.fine_tune(2)

In [84]:
vgg19_bn.lr_find()

In [85]:
classs = ClassificationInterpretation.from_learner(vgg19_bn)
classs.plot_confusion_matrix()

In [86]:
img = PILImage.create(upload.data[-1])
pred,pred_id,probs = vgg19_bn.predict(img)
print(f"Prediction: {pred}")
print(f"Probability: {probs[pred_id]*100:.1f}%")
img

In [87]:
images = !ls "../input/pnevmoniya/test"
images

In [88]:
from tqdm import tqdm
images = !ls "../input/pnevmoniya/test"
prediction = []
numbers = []
for i in tqdm(images):
    number, n_th, probs = resnet50.predict(f"../input/pnevmoniya/test/{i}")
    prediction.append(n_th)
    numbers.append(number)

In [96]:
prediction = []
for num in numbers:
    if num == 'NORMAL':
        prediction.append(0)
    elif num == 'PNEUMONIA':
        prediction.append(1)

In [92]:
submission = pd.read_csv("../input/pnevmoniya/sample_solution.csv")

In [97]:
submission['labels'] = prediction

In [100]:
submission.to_csv("pneumonia_1_0_1.csv",index=False)

In [101]:
pd.read_csv("./pneumonia_1_0_1.csv")

In [102]:
prediction = []
numbers = []
for i in tqdm(images):
    number, n_th, probs = resnet34.predict(f"../input/pnevmoniya/test/{i}")
    prediction.append(n_th)
    numbers.append(number)

In [105]:
prediction2 = []
for num in numbers:
    if num == 'NORMAL':
        prediction2.append(0)
    elif num == 'PNEUMONIA':
        prediction2.append(1)

In [108]:
prediction2[:10]

In [112]:
submission['labels'] = prediction2

In [113]:
submission.to_csv('pneumonia_1_0_2.csv',index=False)